In [2]:
from loading_data import *

/Users/maximeswagel/Library/Mobile Documents/com~apple~CloudDocs/Documents/EPFL/DTU/Courses/SecondSemester/MachineLearning/Assignment2/scripts

Location of the south_african_heart_disease.xls file: /Users/maximeswagel/Library/Mobile Documents/com~apple~CloudDocs/Documents/EPFL/DTU/Courses/SecondSemester/MachineLearning/Assignment2/data/south_african_heart_disease.xls


#### Question 1

The classification problem that we are solving is to predict chd (coronary heart disease) based on the attributes. It is a binary classification problem. The direct interpretation is to know if someone has had a chd based on data. We can extend this to predicting if someone will likely have a chd based on the collected data on this individual. (this is given that the person keeps that same habits) -> maybe this is not applicable because there is an age attribute.
Another use of this classification can be for an insurance company to determine if a client likely has had a chd to then be able to adjust the cost for the insurance.

### Question 2

We have chosen method 2 to be a ANN with one hidden layer. As a complexity-controlling parameter for the logistic regression we consider lambda from the range of 10^(-5) to 10^2. For the ANN, the complexity-controlling parameter that we chose is the number of hidden units in the hidden layer. From beta testing the considered values are [1,2,5,10].

### Question 3

In [4]:
#First we want to normalize and transform our data.

from scipy.stats import zscore

normalised_X = np.copy(X)
#transform
normalised_X[:,6] = np.log(1 + X[:,6]) #add 1 because some alcohol values are 0
#normalise
normalised_X = zscore(normalised_X, axis = 0, ddof = 1)

attributeNames_norm = np.copy(attributeNames)
attributeNames_norm[6] = 'log-alc'
attributeNames_norm = ['normalized ' + attribute for attribute in attributeNames_norm]

#Or without the last binary data

Y = np.copy(normalised_X[:,:-1])
N_y, M_y = Y.shape

attributeNames_y = np.copy(attributeNames_norm[:-1])

In [4]:
import numpy as np
from sklearn import model_selection
from sklearn.dummy import DummyClassifier
import torch
from sklearn.linear_model import LogisticRegression
from dtuimldmtools import train_neural_net

In [39]:
#We need to consider the possible lambdas and the number of hidden units that we want to consider in the inner loop. 
#For the baseline model there is no controlling parameter

#First try
lambda_interval = np.logspace(-5, 2, 100)
n_hidden_units = np.array([1,2,5,10])
# print(n_hidden_units)

K_out = 5
K_in = 5
CV_out = model_selection.KFold(K_out,shuffle=True)
CV_in = model_selection.KFold(K_in,shuffle=True)

# For statistical evaluation : store outer fold predictions for the three models
yhat = []
y_true = []

# For debugging : store inner fold predictions (for each outer fold)
Logistic_full = {}
ANN_full = {}
#where the last coordinate gives us if we are looking at the estimate or the validation values (0 for estimation and 1 for validation)

#The Error for the best model of each type of model in each outer loop
# Train_error = np.zeros((K_out,3))
Test_error= np.zeros((K_out,3))
best_lambda_index = np.int32(np.zeros(K_out))
best_h_index = np.int32(np.zeros(K_out))

for k, (train_index,test_index) in enumerate(CV_out.split(normalised_X,y)):
    print(f"#================OUTER LOOP {k+1}================#")
    #to store the new predictions of the selected model at each outer fold (to then be concatenated in yhat)
    dy = []

    #the training tests for each fold of the outer loop
    X_train = normalised_X[train_index, :]
    y_train = y[train_index]
    X_test = normalised_X[test_index, :]
    y_test = y[test_index]

    #Baseline model
    baseline = DummyClassifier(strategy='most_frequent')
    baseline.fit(X_train,y_train)
    Test_error[k,0] = 1-baseline.score(X_test,y_test)
    dy.append(baseline.predict(X_test))

    #INITIALIZE ERROR HANDLING
    #Error for each model in each of the loops, overwritten at each outer loop 
    Logistic_Inner_test_error= np.zeros((K_in, len(lambda_interval)))
    ANN_Inner_test_error= np.zeros((K_in, len(n_hidden_units)))

    #average error of each model on each outter fold
    Logistic_Model_out_test_error = np.zeros((K_out, len(lambda_interval)))
    ANN_Model_out_test_error = np.zeros((K_out, len(n_hidden_units)))

    #we also need to store the sizes of the folds
    inner_fold_validate_sizes = np.zeros(K_in)

    #Inner Loop
    for i, (Inner_train_index, Inner_test_index) in enumerate(CV_in.split(X_train,y_train)):
        print(f"#================INNER LOOP {k+1}{i+1}================#")
        #initialize the training and validation sets
        X_subtrain = X_train[Inner_train_index]
        y_subtrain = y[Inner_train_index]
        X_validate = X_train[Inner_test_index]
        y_validate = y_train[Inner_test_index]

        #store the size of the validation set
        inner_fold_validate_sizes[i] = X_validate.shape[0]

        #=========#
        #Logistic regression
        #=========#

        #Logistic Regression Model Loop (hyper_parameter tuning)
        print(f"#================Logistic Regression Cross validation================#")
        for s,lamb in enumerate(lambda_interval):
            
            mdl = LogisticRegression(penalty="l2", C=1 / lamb)
            mdl.fit(X_subtrain, y_subtrain)

            # y_subtrain_est = mdl.predict(X_train).T
            y_validate_est = mdl.predict(X_validate).T
            Logistic_Inner_test_error[i, s] = np.sum(y_validate_est != y_validate) / len(y_validate)
            Logistic_full[(i,s,k)] = { 'predictions' : y_validate_est, 'ground_truth' : y_validate}

        #==========#
        #ANN
        #==========#

        #We convert the training and test sets to torch tensors

        X_subtrain = torch.tensor(X_subtrain, dtype=torch.float32)
        y_subtrain = torch.tensor(y_subtrain, dtype=torch.float32).reshape(-1, 1)
        X_validate = torch.tensor(X_validate, dtype=torch.float32)
        y_validate = torch.tensor(y_validate, dtype=torch.float32).reshape(-1, 1)

        #ANN cross validation loop
        for j, n in enumerate(n_hidden_units):
            print(f"#================ANN Model {k+1}{i+1}{j+1}================#")
            # The lambda-syntax defines an anonymous function, which is used here to
            # make it easy to make new networks within each cross validation fold
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n),  # M features to H hiden units
                # 1st transfer function, either Tanh or ReLU:
                torch.nn.Tanh(),  
                # torch.nn.ReLU(),
                torch.nn.Linear(n, 1),  # H hidden units to 1 output neuron
                torch.nn.Sigmoid(),  # final tranfer function
            )
            #Loss function (Binary Cross Entropy)
            loss_fn = torch.nn.BCELoss()
            # Train for a maximum of 10000 steps, or until convergence
            max_iter = 10000
        
            net, final_loss, learning_curve = train_neural_net(
            model, loss_fn, X=X_subtrain, y=y_subtrain, n_replicates=3, max_iter=max_iter
            )
            y_validate_est = net(X_validate).detach().numpy()
            y_validate_pred = (y_validate_est > 0.5).astype(int)
            y_validate_np = y_validate.numpy()
            ANN_Inner_test_error[i, j] = np.sum(y_validate_pred != y_validate) / len(y_validate)
            ANN_full[(i,j,k)] = { 'predictions' : y_validate_pred.squeeze(), 'ground_truth' : y_validate_np.squeeze()}

    #Average Model Error calculation for Regression   
    Logistic_Model_out_test_error = np.sum(inner_fold_validate_sizes[:,None]*Logistic_Inner_test_error, axis = 0)/X_train.shape[0]
    best_lambda_index[k] = int(np.argmin(Logistic_Model_out_test_error))
    #Average Model Error calculation for ANN   
    ANN_Model_out_test_error = np.sum(inner_fold_validate_sizes[:,None]*ANN_Inner_test_error, axis = 0)/X_train.shape[0]
    best_h_index[k] = int(np.argmin(ANN_Model_out_test_error))

    #Retrain the best model on the full X_train for regression
    mdl = LogisticRegression(penalty="l2", C = 1/lambda_interval[best_lambda_index[k]])
    mdl.fit(X_train,y_train)
    y_test_est = mdl.predict(X_test).T
    Test_error[k,1] = np.sum(y_test_est != y_test) / len(y_test)
    
    #to store the predictions
    dy.append(y_test_est.T)

    #Retrain the best ANN model
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
    model = lambda: torch.nn.Sequential(
        torch.nn.Linear(M, n_hidden_units[best_h_index[k]]),  # M features to H hiden units
        # 1st transfer function, either Tanh or ReLU:
        # torch.nn.Tanh(),  
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden_units[best_h_index[k]], 1),  # H hidden units to 1 output neuron
        torch.nn.Sigmoid(),  # final tranfer function
    )
    #Loss function (Binary Cross Entropy)
    loss_fn = torch.nn.BCELoss()
    # Train for a maximum of 10000 steps, or until convergence
    max_iter = 10000
        
    net, final_loss, learning_curve = train_neural_net(
        model, loss_fn, X=X_train, y=y_train, n_replicates=3, max_iter=max_iter
    )
    y_test_est = net(X_test).detach().numpy()
    y_test_pred = (y_test_est > 0.5).astype(int)
    y_test = y_test.numpy()
    Test_error[k, 2] = np.sum(y_test_pred != y_test) / len(y_test)


    #to store the predictions
    dy.append(y_test_pred.squeeze())
    dy = np.stack(dy, axis=1)
    yhat.append(dy)
    y_true.append(y_test)

yhat = np.array(np.concatenate(yhat))
y_true = np.array(np.concatenate(y_true)).squeeze()

#================OUTER LOOP 1================#
#================INNER LOOP 11================#
#================Logistic Regression Cross validation================#
#================ANN Model 111================#

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.5944429	5.634843e-05
		2000	0.58776647	1.5211285e-06
		3000	0.58645487	3.150691e-06
		4000	0.5841856	4.3872847e-06
		5000	0.5814483	4.8179772e-06
		6000	0.5785843	4.7388094e-06
		7000	0.57604647	3.828447e-06
		8000	0.5740531	3.218758e-06
		9000	0.57254225	2.0821003e-06
		10000	0.5714255	1.564628e-06
		Final loss:
		10000	0.5714255	1.564628e-06

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.5967805	3.2858454e-05
		2000	0.58400345	5.4092643e-06
		3000	0.5807568	5.952662e-06
		4000	0.57715565	7.2290636e-06
		5000	0.5737881	4.6745417e-06
		6000	0.5715577	3.4413788e-06
		7000	0.56988174	2.9285466e-06
		8000	0.5685235	2.096818e-06
		9000	0.567394	1.9959427e-06
		10000	0.56640285	1.5785024e-06
		Final loss:
		10000	0.56640285	1.57

In [40]:
print(f'Optimal lambda : {lambda_interval[best_lambda_index]}')
print(f'Optimal hidden neuron number : {n_hidden_units[best_h_index]}' )
print(Test_error)

Optimal lambda : [23.101297   27.18588243 16.68100537 31.99267138 16.68100537]
Optimal hidden neuron number : [1 1 1 1 1]
[[0.38709677 0.27956989 0.27956989]
 [0.34408602 0.23655914 0.27956989]
 [0.27173913 0.27173913 0.29347826]
 [0.36956522 0.29347826 0.29347826]
 [0.35869565 0.25       0.25      ]]


In [41]:
#The order is baseline/logistic/ANN

Briefly we can say that the logistic regression seems to be the best model and that both model seem better then the baseline. We see that the optimal number of hidden units for the ANN is always 1, which is more or less equivalent to the logistic regression. This hints at the fact that there is probably no statistically significant difference between our ANN and the logisitic regression? Maybe also we can say that given the fairly high value of the regression controlling parameters, and the underperformance of the ANN (at least I expected more) there is a high risk of overfitting in the problem. This hints towards the fact that we would need more data or more significant data to lower the inherent variance of the problem.

### Question 4

In [42]:
from dtuimldmtools import mcnemar
#we suppose that we have the y_hat and y_true for all models.

# Compute the Jeffreys interval
alpha = 0.05
[thetahatA, CIA, p] = mcnemar(y_true, yhat[:, 0], yhat[:, 1], alpha=alpha)
[thetahatB, CIB, p] = mcnemar(y_true, yhat[:, 1], yhat[:, 2], alpha=alpha)
[thetahatC, CIB, p] = mcnemar(y_true, yhat[:, 0], yhat[:, 2], alpha=alpha)

Result of McNemars test using alpha= 0.05
Comparison matrix n
[[263.  39.]
 [ 76.  84.]]
Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] =  (-0.12485715521577767, -0.035155970330023156)
p-value for two-sided test A and B have same accuracy (exact binomial test): p= 0.0007171910479201684
Result of McNemars test using alpha= 0.05
Comparison matrix n
[[323.  16.]
 [ 10. 113.]]
Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] =  (-0.00859144752307639, 0.03455950911849737)
p-value for two-sided test A and B have same accuracy (exact binomial test): p= 0.32693958282470703
Result of McNemars test using alpha= 0.05
Comparison matrix n
[[253.  49.]
 [ 80.  80.]]
Approximate 1-alpha confidence interval of theta: [thetaL,thetaU] =  (-0.11475981409768699, -0.01928770584052919)
p-value for two-sided test A and B have same accuracy (exact binomial test): p= 0.008007673208090866


We chose to apply the McNemar's test. Therefore we are in setting 1 and we can only discuss the differences in the models on the given data set. We can conclude that the regression and the ANN are statistically significantly better than the classifier, howevert the logistic regression is not statistically better the ANN classifier. 

### Question 5

To yield a control parameter for the logistic regression model we could take the parameter with the lowest error or we could also take the average of all the coefficients yielded by all the models to yield a model (but not directly a controlling parameter). Here we will take the lambda with the lowest estimated generalizing error.

In [43]:
final_lambda_index = np.argmin(Test_error[:,1])
final_lambda = (lambda_interval[best_lambda_index])[final_lambda_index]
print(final_lambda)

27.1858824273294


In [44]:
#Now we train the final model
final_model = LogisticRegression(penalty="l2", C=1 / final_lambda, max_iter = 1000)
final_model.fit(X, y)
print(final_model.coef_)

[[ 0.00593957  0.07360055  0.16702645  0.01757337  0.03944127 -0.05584428
   0.00083319  0.04812059  0.38904451]]


Here the real relevant features are the 3rd one and the last one.

In [48]:
#3rd one
print(attributeNames_norm[2])

#last one
print(attributeNames_norm[-1])

normalized ldl
normalized famhist


In [5]:
print(attributeNames_norm)

['normalized sbp', 'normalized tobacco', 'normalized ldl', 'normalized adiposity', 'normalized typea', 'normalized obesity', 'normalized log-alc', 'normalized age', 'normalized famhist']


Others may also be considered significant... (>0.1 maybe ?)

### Discussion

In the classification part of the report we have discovered that the data is not informative enough to yield a satisfying prediction of the history of chd in a patient. In other words, the inherent variance of the problem is too large. Although we have been able to make better predictions then the baseline, further exploration in methods and data preprocessing are needed. One could try for example to fit deep neural networks. While this might sound counterintuitive since our problem is very prone to overfitting and that deep neural network seem more complicated than the 1 layer ANN considered here, it may discover new relationships in the data that our simple models can't catch - simple but hidden relationships.